###  <font color = "orangered">  Emparchar los datasets no resueltos de manera automatizada por v2_renaming </font>

<b> <font color = "bisque"> dfProblemas_tipo2 </font> </b>

<b> <font color = "gold"> En el tópico DESHUM el problema de no lectura geocódigos se da porque el csv tiene una columna con geodódigos del tipo 'ZZA.VHHD' y en la columna geocodigo de desc_fundar están como 'DESHUM_ZZA.VHHD' </font> </b>

In [2]:
import os
import pandas as pd
import unicodedata

# funcion eliminar tildesr
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return "".join(c for c in nkfd if not unicodedata.combining(c))

# dir DESHUM nuevo 
DESHUM_DIR = '/home/lucia/Desktop/Fundar/data_renaming/new_data_argendataMayo/DESHUM'

# geonomenclador
df_geonomenclador = pd.read_csv('/home/lucia/Desktop/Fundar/data_renaming/input' + '/geonomenclador_PostPesca.csv')

# clave = geocodigo normalizado con (name_long, desc_fundar)
geo_by_code = {}
for _, row in df_geonomenclador.iterrows():
    raw_code   = str(row['geocodigo'])
    code_norm  = sin_tildes(raw_code.strip().upper())
    name_long  = row['name_long']
    desc_fund  = row['desc_fundar']
    geo_by_code[code_norm] = (name_long, desc_fund)

# proceso cada csv de new_data_argendataMayo/DESHUM
for archivo in os.listdir(DESHUM_DIR):
    if not archivo.lower().endswith('.csv'):
        continue
    
    path_csv = os.path.join(DESHUM_DIR, archivo)
    df = pd.read_csv(path_csv, encoding='utf-8', delimiter=',')
    
    # si no tiene la columna 'iso3', continuo (en DESHUM todas lo tienen por lo que me fijé, pero lo veo por las dudas)
    if 'iso3' not in df.columns:
        continue
    
    # miro que estén las columnas geocodigoFundar y geonombreFundar
    if 'geocodigoFundar' not in df.columns:
        df['geocodigoFundar'] = pd.NA
    if 'geonombreFundar' not in df.columns:
        df['geonombreFundar'] = pd.NA
    
    # recorro cada fila del csv y ejecuto acción si si ambas columnas geocodigoFundar y geonombreFundar están vacías o NaN
    for idx in df.index:
        gcf = df.at[idx, 'geocodigoFundar']
        gnf = df.at[idx, 'geonombreFundar']

        # Verificar si ambas columnas están vacías (NaN o None)
        if pd.isna(gcf) and pd.isna(gnf):
            raw_iso3 = str(df.at[idx, 'iso3']).strip().upper()
            if raw_iso3 == '':
                continue  # no hay iso3, no sigue (en DESHUM por lo que vi todos los csv tienen iso 3, pero por las dudas)

            # genero el código con prefijo 'DESHUM'
            nuevo_code = f"DESHUM_{raw_iso3}"
            norm_code = sin_tildes(nuevo_code)

            # si existe en geonomenclador, completar ambas columnas
            if norm_code in geo_by_code:
                nl, desc0 = geo_by_code[norm_code]
                df.at[idx, 'geocodigoFundar'] = norm_code
                df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
            # si no existe en geonomenclador, dejar NaN

    # guardar csv modificado en su correspondiente topico, con UTF-8 y separador coma
    df.to_csv(path_csv, index=False, encoding='utf-8', sep=',')



<b> <font color = "bisque"> filtro los problemas ya resueltos sobre el tópico DESHUM que afectaba tanto a dfProblemas_tipo2 como a dfProblemas_tipo3, y dejo los problemas restantes para verlos en una segunda vuelta  </font> </b>

In [ ]:
import os
import pandas as pd

# carpeta con dfProblemas_tipo2 como a dfProblemas_tipo3 y donde guardo los nuevos
base_path = '/home/lucia/Desktop/Fundar/data_renaming/problematicas_renaming'

# nombres archivos
file_tipo2 = os.path.join(base_path, 'dfProblemas_tipo2.csv')
file_tipo3 = os.path.join(base_path, 'dfProblemas_tipo3.csv')

# cargar dfProblemas_tipo2 y filtrar filas donde topicos == 'DESHUM', y los guardo como dfProblemas_tipo2_2daVuelta.csv
df_tipo2 = pd.read_csv(file_tipo2)
df_tipo2_filtrado = df_tipo2[df_tipo2['TOPICO'] != 'DESHUM']

output_tipo2 = os.path.join(base_path, 'dfProblemas_tipo2_2daVuelta.csv')
df_tipo2_filtrado.to_csv(output_tipo2, index=False, encoding='utf-8', sep=',')

df_tipo2_filtrado

,TOPICO,csv,problema
0,ESTPRO,va_sectorial_pais.csv,dataset tiene el codigo 'TZ1' que NO esta incl...
1,ESTPRO,particip_va_intensivos_id_bys.csv,dataset tiene el codigo 'EU28XEU15' que NO est...
2,ESTPRO,particip_va_intensivos_id_bys.csv,dataset tiene el codigo 'NAFTA' que NO esta in...
3,ESTPRO,particip_va_intensivos_id_bys.csv,dataset tiene el codigo 'S2_S8' que NO esta in...
4,ESTPRO,va_sectorial_pais_bys.csv,dataset tiene el codigo 'TZ1' que NO esta incl...
5,COMEXT,exportaciones_servicios_top_20_destinos.csv,dataset tiene el codigo 'NAN' que NO esta incl...
55,ACECON,A4_efecto_bs.csv,dataset tiene el codigo 'ZAR' que NO esta incl...
56,ACECON,2_pibpc_salud_edu.csv,dataset tiene el codigo 'ZAR' que NO esta incl...
57,ACECON,A1_inb_pib.csv,dataset tiene el codigo 'TMP' que NO esta incl...
58,ACECON,A5_pibpc_propeeuu_ppa_usd.csv,dataset tiene el codigo 'ZAR' que NO esta incl...


In [ ]:
# carpeta con dfProblemas_tipo2 como a dfProblemas_tipo3 y donde guardo los nuevos
base_path = '/home/lucia/Desktop/Fundar/data_renaming/problematicas_renaming'

# nombres archivos
file_tipo2 = os.path.join(base_path, 'dfProblemas_tipo2.csv')
file_tipo3 = os.path.join(base_path, 'dfProblemas_tipo3.csv')

# cargar dfProblemas_tipo3 y filtrar filas donde topicos == 'DESHUM', y los guardo como dfProblemas_tipo3_2daVuelta.csv
df_tipo3 = pd.read_csv(file_tipo3)
df_tipo3_filtrado = df_tipo3[df_tipo3['TOPICO'] != 'DESHUM']

output_tipo3 = os.path.join(base_path, 'dfProblemas_tipo3_2daVuelta.csv')
df_tipo3_filtrado.to_csv(output_tipo3, index=False, encoding='utf-8', sep=',')

df_tipo3_filtrado

,TOPICO,csv,problema
0,SEBACO,22_participacion_expo_sbc.csv,dataset tiene el codigo 'CHT' que NO esta incl...
1,SEBACO,22_participacion_expo_sbc.csv,dataset tiene el codigo 'ROM' que NO esta incl...
2,SEBACO,23_participacion_expo_ssi.csv,dataset tiene el codigo 'ROM' que NO esta incl...
3,COMEXT,composicion_exportaciones_bienes_sectores_Bram...,dataset tiene el codigo 'S19' que NO esta incl...
4,COMEXT,composicion_importaciones_bienes_sectores_Bram...,dataset tiene el codigo 'S19' que NO esta incl...
5,TRANEN,matriz_prim_mundo_historic.csv,dataset tiene el codigo 'OWID_WRL' que NO esta...
6,TRANEN,intensidad_energ_mundo.csv,dataset tiene el codigo 'OWID_WRL' que NO esta...
7,TRANEN,intensidad_energ_mundo.csv,dataset tiene el codigo 'OWID_CZS' que NO esta...
8,TRANEN,intensidad_energ_mundo.csv,dataset tiene el codigo 'OWID_YGS' que NO esta...
9,TRANEN,generacion_nuclear_twh.csv,dataset tiene el codigo 'OWID_WRL' que NO esta...


In [7]:
# veamos como son los dfProblemas_tipo4.csv

# carpeta donde con dfProblemas_tipo4 
base_path = '/home/lucia/Desktop/Fundar/data_renaming/problematicas_renaming'

file_tipo4 = os.path.join(base_path, 'dfProblemas_tipo4.csv')

# cargar dfProblemas_tipo4 
df_tipo4 = pd.read_csv(file_tipo4)

df_tipo4

,TOPICO,csv,problema
0,SEBACO,06_empleo_sbc_provincia.csv,Para la string 'CABA' se debería generar un co...
1,SEBACO,06_empleo_sbc_provincia.csv,Para la string 'GBA' se debería generar un cod...
2,ESTPRO,densidad_empresarial_depto.csv,Para la string 'CIUDAD DE BUENOS AIRES' se deb...
3,ESTPRO,densidad_empresarial_depto.csv,Para la string 'NAN' se debería generar un cod...
4,ESTPRO,densidad_nbi.csv,Para la string 'CIUDAD DE BUENOS AIRES' se deb...
5,ESTPRO,densidad_nbi.csv,Para la string 'NAN' se debería generar un cod...
6,MERTRA,tasa_empleo_por_franja_etaria_anio_provincia.csv,Para la string 'CABA' se debería generar un co...
7,MERTRA,tasa_empleo_por_franja_etaria_anio_provincia.csv,Para la string 'TOTAL' se debería generar un c...
8,MERTRA,tasa_empleo_menores_provincia.csv,Para la string 'CIUDAD DE BUENOS AIRES' se deb...
9,MERTRA,tasa_empleo_menores_provincia.csv,Para la string 'TOTAL' se debería generar un c...
